# Final Project

In [10]:
import os
import json
import time
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener

try:
    consumer_key = 'uQ9FRsAAUMuJysilmgxq5VlMS'
    consumer_secret = 'gMLbtAIzDusabJ3zmaIuoXDYlbnPau6fqSkwjLdkSRdOCwa4a8'
    access_token = '784558015030632448-NHHZm5u67bRRMie9q3qkWHSPytrcRXu'
    access_secret = 'Fu5lFvFktrNe2k364q6xocV2lUEUcfwuXWau2qQS3Xuop'

    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)

    api = tweepy.API(auth)
except:
    print("Error: Authentication Failed")
    
#The Streaming API    
start_time = time.time() #grabs the system time
keyword_list = ['#RoyalChallengers','#GujratLions'] #track list
#Listner Class Override
class listener(StreamListener):
    
    def __init__(self, start_time, time_limit=60):
        self.time = start_time
        self.limit = time_limit
        self.tweet_data = []
        
    def on_data(self, data):
        saveFile = open('raw_tweets_static.json', 'a+', encoding='utf-8')
        while (time.time() - self.time) < self.limit:
            try:
                self.tweet_data.append(data)
                return True
            except BaseException as e:
                print('failed ondata', str(e))
                time.sleep(5)
                pass
        saveFile = open('raw_tweets_live.json', 'w', encoding='utf-8')
        saveFile.write(u'[\n')
        saveFile.write(','.join(self.tweet_data))
        saveFile.write(u'\n]')
        saveFile.close()
        exit()
    def on_error(self, status):
        print (status)
        return True
        
twitterStream = Stream(auth, listener(start_time, time_limit=30))#initialize Stream object with a time out limit
twitterStream.filter(track=keyword_list,languages=['en']) #call the filter method to run the Stream Object

#REST tweepy API
def process_or_store(tweet):
    with open('raw_tweets_static.json', 'a+',encoding='utf-8') as f:
        f.write(json.dumps(tweet))
        f.write("\n")
        f.close()
        
for tweet in tweepy.Cursor(api.search, since='2017-04-22', until='2017-04-28', q='GujratLions',language=['en']).items(1000):
       process_or_store(tweet._json) 
       